In [2]:
import json
import numpy as np
import pandas as pd
import time

# Fungsi aktivasi
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

# Fungsi update SGD
def sgd_update(weight, gradient, learning_rate):
    return weight + learning_rate * gradient

# Fungsi update Momentum
def momentum_update(weight, gradient, learning_rate, velocity, momentum):
    velocity = momentum * velocity + learning_rate * gradient
    return weight + velocity, velocity

# Fungsi update RMSProp
def rmsprop_update(weight, gradient, v, learning_rate, beta2, epsilon=1e-8):
    v = beta2 * v + (1 - beta2) * (gradient ** 2)
    weight -= learning_rate * gradient / (np.sqrt(v) + epsilon)
    return weight, v

# Fungsi update Adam
def adam_update(weight, gradient, m, v, t, learning_rate, beta1, beta2, epsilon=1e-8):
    m = beta1 * m + (1 - beta1) * gradient
    v = beta2 * v + (1 - beta2) * (gradient ** 2)
    m_hat = m / (1 - beta1 ** t)
    v_hat = v / (1 - beta2 ** t)
    weight += learning_rate * m_hat / (np.sqrt(v_hat) + epsilon)
    return weight, m, v



# Neural network class
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, optimizer='sgd'):
        # Inisialisasi bobot dan bias
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)
        self.weights_hidden_output = np.random.randn(hidden_size, output_size)
        self.bias_hidden = np.random.randn(hidden_size)
        self.bias_output = np.random.randn(output_size)

        # Setup variabel terkait optimizer
        self.optimizer = optimizer
        self.velocity_ih = np.zeros_like(self.weights_input_hidden)
        self.velocity_ho = np.zeros_like(self.weights_hidden_output)
        self.m_ih = np.zeros_like(self.weights_input_hidden)
        self.v_ih = np.zeros_like(self.weights_input_hidden)
        self.m_ho = np.zeros_like(self.weights_hidden_output)
        self.v_ho = np.zeros_like(self.weights_hidden_output)
        self.beta1, self.beta2 = 0.9, 0.999
        self.t = 1

    def forward(self, inputs):
        # Propagasi maju untuk batch
        self.hidden_input = np.dot(inputs, self.weights_input_hidden) + self.bias_hidden
        self.hidden_output = sigmoid(self.hidden_input)
        self.output_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output
        self.predicted_output = sigmoid(self.output_input)
        return self.predicted_output

    def backward(self, inputs, targets, learning_rate):
        # Backpropagation untuk batch
        error = targets - self.predicted_output
        delta_output = error * sigmoid_derivative(self.predicted_output)
        error_hidden = delta_output.dot(self.weights_hidden_output.T)
        delta_hidden = error_hidden * sigmoid_derivative(self.hidden_output)

        # Update bobot menggunakan optimizer yang dipilih
        if self.optimizer == 'sgd':
            self.weights_hidden_output = sgd_update(self.weights_hidden_output,
                                                     np.dot(self.hidden_output.T, delta_output), learning_rate)
            self.weights_input_hidden = sgd_update(self.weights_input_hidden,
                                                    np.dot(inputs.T, delta_hidden), learning_rate)

        elif self.optimizer == 'momentum':
            self.weights_hidden_output, self.velocity_ho = momentum_update(
                self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
                learning_rate, self.velocity_ho, momentum=0.9
            )
            self.weights_input_hidden, self.velocity_ih = momentum_update(
                self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
                learning_rate, self.velocity_ih, momentum=0.9
            )

        elif self.optimizer == 'adam':
            self.weights_hidden_output, self.m_ho, self.v_ho = adam_update(
                self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
                self.m_ho, self.v_ho, self.t, learning_rate, self.beta1, self.beta2
            )
            self.weights_input_hidden, self.m_ih, self.v_ih = adam_update(
                self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
                self.m_ih, self.v_ih, self.t, learning_rate, self.beta1, self.beta2
            )
            self.t += 1

        elif self.optimizer == 'rmsprop':
            self.weights_hidden_output, self.v_ho = rmsprop_update(
                self.weights_hidden_output, np.dot(self.hidden_output.T, delta_output),
                self.v_ho, learning_rate, self.beta2
            )
            self.weights_input_hidden, self.v_ih = rmsprop_update(
                self.weights_input_hidden, np.dot(inputs.T, delta_hidden),
                self.v_ih, learning_rate, self.beta2
            )

    def train(self, training_data, targets, epochs, learning_rate, batch_size):
        error_history = []  # Untuk menyimpan error di setiap epoch
        for epoch in range(epochs):
            total_error = 0
            for i in range(0, len(training_data), batch_size):
                # Buat batch
                batch_inputs = training_data[i:i + batch_size]
                batch_targets = targets[i:i + batch_size].reshape(-1, 1)

                # Forward pass
                self.forward(batch_inputs)

                # Backward pass
                self.backward(batch_inputs, batch_targets, learning_rate)

                # Hitung error untuk batch
                total_error += np.mean(np.square(batch_targets - self.predicted_output))

            error_history.append(total_error / (len(training_data) / batch_size))

            # Print error setiap 10 epoch
            if epoch % 10 == 0:
                print(f"Epoch {epoch}, Error: {total_error / (len(training_data) / batch_size)}")
        # return error_history

    def predict(self, inputs):
        return self.forward(inputs)



### hidden = 8
### epoch = 100
### batch size = 10
### optimizer = adam

### disini saya mencoba untuk memperbanyak hidden layer yang saya gunakan, yaitu menjadi 8 hidden layer dengan menggunakan optimizer yang sama yaitu adam. output yang didapatkan menghasilkan error yang terus mengecil pada setiap epoch. tetapi error yang dihasilkan lebih besar dibandingkan dengan menggunakan hidden layer 4

In [20]:
url = 'https://raw.githubusercontent.com/mirohmi/Heart_Disease_Diagnose/refs/heads/master/heart_diseases.csv'
data = pd.read_csv(url)

#data = pd.read_csv("heart_diseases.csv")
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 8  # Jumlah node di hidden layer
output_size = 1  # Jumlah output
learning_rate = 0.01
epochs = 100
batch_size = 10

nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='adam')
nn.train(training_data, targets, epochs, learning_rate, batch_size)

Epoch 0, Error: 0.41614975128345855
Epoch 10, Error: 0.2390979920297552
Epoch 20, Error: 0.24923084538298784
Epoch 30, Error: 0.24950467393817838
Epoch 40, Error: 0.2494943061376514
Epoch 50, Error: 0.24938525058012947
Epoch 60, Error: 0.24929893538678946
Epoch 70, Error: 0.24923157461791356
Epoch 80, Error: 0.24917919671585126
Epoch 90, Error: 0.2491383683559075


In [25]:
url = 'https://raw.githubusercontent.com/mirohmi/Heart_Disease_Diagnose/refs/heads/master/heart_diseases.csv'
data = pd.read_csv(url)

#data = pd.read_csv("heart_diseases.csv")
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 8  # Jumlah node di hidden layer
output_size = 1  # Jumlah output
learning_rate = 0.01
epochs = 100
batch_size = 1000

nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='adam')
nn.train(training_data, targets, epochs, learning_rate, batch_size)

Epoch 0, Error: 1.2175209783878436
Epoch 10, Error: 0.9738518464877384
Epoch 20, Error: 0.9132411207313392
Epoch 30, Error: 0.8735838004994113
Epoch 40, Error: 0.8567695734664467
Epoch 50, Error: 0.8426215358178044
Epoch 60, Error: 0.828232584173308
Epoch 70, Error: 0.8110102322468495
Epoch 80, Error: 0.7912176458937606
Epoch 90, Error: 0.7627870683610734


<ipython-input-2-a25847be59d5>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


In [18]:
# Test network
for inputs in training_data:
    prediction = nn.predict(inputs.reshape(1, -1))  # Ubah input ke bentuk batch
    print(f"Predicted Output: {prediction}")




# Fungsi untuk melatih dengan optimizer tertentu
def train_with_optimizer(optimizer, input_size, hidden_size, output_size,
                         training_data, targets, epochs, learning_rate, batch_size):
    # Pilih optimizer langsung di sini
    nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='adam')
    start_time = time.time()
    error_history = nn.train(training_data, targets, epochs, learning_rate, batch_size)
    training_time = time.time() - start_time
    return error_history, training_time

for inputs in training_data:
    prediction = nn.predict(inputs)
    print(f"Input: {inputs}, Predicted Output: {prediction}")


# Latih model dengan berbagai optimizer
optimizers = ['sgd', 'adam']
results = {}

for opt in optimizers:
    print(f"Training with {opt} optimizer...")
    error_history, training_time = train_with_optimizer(
        opt, input_size, hidden_size, output_size,
        training_data, targets, epochs, learning_rate, batch_size
    )
    results[opt] = {
        'error_history': error_history,
        'training_time': training_time
    }

# Simpan hasil ke file JSON
with open('optimizer_results_single_line.json', 'w') as f:
    for optimizer, data in results.items():
        json_line = json.dumps({optimizer: data}, separators=(',', ':'))
        f.write(json_line + '\n')

print("Results saved to optimizer_results_single_line.json")

Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted Output: [[0.45082031]]
Predicted 

<ipython-input-2-a25847be59d5>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))



Input: [ 44.   1.   4. 110. 197.   0.   2. 177.   0.   0.   1.   1.   3.], Predicted Output: [0.45082031]
Input: [ 65.    0.    3.  160.  360.    0.    2.  151.    0.    0.8   1.    0.
   3. ], Predicted Output: [0.45082031]
Input: [ 60.    1.    4.  125.  258.    0.    2.  141.    1.    2.8   2.    1.
   7. ], Predicted Output: [0.45082031]
Input: [ 51.    0.    3.  140.  308.    0.    2.  142.    0.    1.5   1.    1.
   3. ], Predicted Output: [0.45082031]
Input: [4.80e+01 1.00e+00 2.00e+00 1.30e+02 2.45e+02 0.00e+00 2.00e+00 1.80e+02
 0.00e+00 2.00e-01 2.00e+00 0.00e+00 3.00e+00], Predicted Output: [0.45082031]
Input: [ 68.    1.    3.  180.  274.    1.    2.  150.    1.    1.6   2.    0.
   7. ], Predicted Output: [0.45082031]
Input: [5.20e+01 1.00e+00 2.00e+00 1.20e+02 3.25e+02 0.00e+00 0.00e+00 1.72e+02
 0.00e+00 2.00e-01 1.00e+00 0.00e+00 3.00e+00], Predicted Output: [0.45082031]
Input: [ 44.   1.   3. 140. 235.   0.   2. 180.   0.   0.   1.   0.   3.], Predicted Output: [0.450

## Latihan Mandiri

### hidden = 4
### epoch = 100
### batch size = 10
### optimizer = adam

### output yang dihasilkan adalah pada setiap epoch error yang didapatkan semakin kecil

In [11]:
url = 'https://raw.githubusercontent.com/mirohmi/Heart_Disease_Diagnose/refs/heads/master/heart_diseases.csv'
data = pd.read_csv(url)

#data = pd.read_csv("heart_diseases.csv")
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 4  # Jumlah node di hidden layer
output_size = 1  # Jumlah output
learning_rate = 0.01
epochs = 100
batch_size = 10

nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='adam')
nn.train(training_data, targets, epochs, learning_rate, batch_size)


<ipython-input-2-a25847be59d5>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 0.3149901556293327
Epoch 10, Error: 0.2531418228980655
Epoch 20, Error: 0.25231886587921776
Epoch 30, Error: 0.2518884921930473
Epoch 40, Error: 0.25166051801907685
Epoch 50, Error: 0.25155466975088375
Epoch 60, Error: 0.25150733884032694
Epoch 70, Error: 0.25148602611117965
Epoch 80, Error: 0.25147632758968547
Epoch 90, Error: 0.2514719562638033


### hidden = 4
### epoch = 100
### batch size = 10
### optimizer = sgd

### disini saya mencoba untuk menggunakan oprimizer yang berbeda yaitu sgd. dari hasil perbandingan dengan menggunaan adam dan sgd didapatkanlah perbandingan bahwa output yang dihasilkan oleh sgd lebih kecil dibandingkan dengan menggunakan adam.

In [15]:
url = 'https://raw.githubusercontent.com/mirohmi/Heart_Disease_Diagnose/refs/heads/master/heart_diseases.csv'
data = pd.read_csv(url)

#data = pd.read_csv("heart_diseases.csv")
training_data = data.iloc[:, :-1].values
targets = data.iloc[:, -1].values

# Parameter jaringan saraf
input_size = training_data.shape[1]
hidden_size = 4  # Jumlah node di hidden layer
output_size = 1  # Jumlah output
learning_rate = 0.01
epochs = 100
batch_size = 10

nn = NeuralNetwork(input_size, hidden_size, output_size, optimizer='sgd')
nn.train(training_data, targets, epochs, learning_rate, batch_size)

<ipython-input-2-a25847be59d5>:8: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 0, Error: 0.26537451532890516
Epoch 10, Error: 0.25218847680279577
Epoch 20, Error: 0.25147399363492057
Epoch 30, Error: 0.25142019073548116
Epoch 40, Error: 0.25140321250893416
Epoch 50, Error: 0.2513887822430149
Epoch 60, Error: 0.2513736831103566
Epoch 70, Error: 0.2513576918681557
Epoch 80, Error: 0.2513412063955576
Epoch 90, Error: 0.25132469679239644
